<a href="https://colab.research.google.com/github/MahithaPoduvu/Mahitha_INFO5731_-Spring2023/blob/main/In_class_exercise_03_02282023_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The third In-class-exercise (2/28/2023, 40 points in total)

The purpose of this exercise is to understand text representation.

Question 1 (10 points): Describe an interesting text classification or text mining task and explain what kind of features might be useful for you to build the machine learning model. List your features and explain why these features might be helpful. You need to list at least five different types of features.

In [ ]:

'''
The text classification task chosen here is Sentiment analysis Tweets on Avatar movie. For this task, we want to build a machine learning model that can classify tweets as either positive or negative based on the sentiment expressed in the text.
Some features that might be useful for this task include:

1.Bag of words features: This represents the text as a set of words without any regard to the order of words in the text. Counting the frequency of each word in a tweet and using this as a feature for the model. This can capture important keywords that are associated with positive or negative sentiment.

2.Part-of-speech features: Analyzing the syntactic structure of the text by identifying the part of speech of each word like noun, verb, adjective and using those tags as features. 

3.Sentiment lexicons: Using pre-built lists of words that are annotated with their sentiment polarity (positive, negative, or neutral) and counting the frequency of those words in the text. This can capture the use of words that are strongly associated with positive or negative sentiment.

4.N-grams: Analyzing sequences of words of length n (e.g. bigrams, trigrams) and using those sequences as features. This can capture more complex patterns of language.

5.Emotion features: To capture the emotional content in the text such as anger, fear, joy, sadness etc. These features can be useful for sentiment analysis as different emotions may indicate different sentiment.

6.words embedding: Used to capture the context and meaning of the words in the text.

'''

In [ ]:
!pip install snscrape   # since it is installing every time i had made it to comment
import snscrape.modules.twitter as sntwitter
import snscrape.modules.twitter as twitterScrapper
import pandas as pd


tweet_data = [] #creating the emppty list
number = 100   # for scrapping 100 tweets
username = input() # to prompt user input , here I have scraped @officialavatar tweets
d = f" from:{username}" # defining d as f from user name and to use it in loop

for i, tweets in enumerate(sntwitter.TwitterSearchScraper(d).get_items()):
  if i > number:
    break
  tweet_data.append({tweets.rawContent,tweets.user.username})
  print(tweet_data)

# to create the data frame
df = pd.DataFrame(tweet_data,columns=['tweets','username'])
print(df)
from google.colab import drive
drive.mount('/content/drive')
df.to_csv("//content//drive//MyDrive//tweet_data.csv")
data = pd.read_csv("//content//drive//MyDrive//tweet_data.csv")
print(data)

Question 2 (10 points): Write python code to extract these features you discussed above. You can collect a few sample text data for the feature extraction. 

In [ ]:
data = pd.read_csv("//content//drive//MyDrive//tweet_data.csv")
print(data.head)
data.info # to get the info of the data


In [3]:
#Importing libaries
import warnings
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import string
import nltk
%matplotlib inline
warnings.filterwarnings('ignore')

# for preprocessing data
# removes pattern in the input text
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for word in r:
        input_txt = re.sub(word, "", input_txt)
    return input_txt
#data.head()
#Lowercase all texts
data['tweets'] = data['tweets'].str.lower()
data.head
# remove twitter handles (@user)
data['clean_tweets'] = np.vectorize(remove_pattern)(data['tweets'], "@[\w]*")
#data.head
# remove special characters, numbers and punctuations
data['clean_tweets'] = data['clean_tweets'].str.replace("[^a-zA-Z#]", " ")
#data.head()
# remove short words less tha 3 characters long
data['clean_tweets'] = data['clean_tweets'].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))
#data.head()


In [ ]:
#sentiment lexicon
#!pip install TextBlob
from textblob import TextBlob

def getTextSubjectivity(tokenized_tweet):
  return TextBlob(tokenized_tweet).sentiment.subjectivity
#to get text polarity
def getTextPolarity(tokenized_tweet):
  return TextBlob(tokenized_tweet).sentiment.polarity

data_subjectivity = data['clean_tweets'].apply(getTextSubjectivity)
data_polarity = data['clean_tweets'].apply(getTextSubjectivity)
#print(data_subjectvity,data_polarity)

def getTextAnalysis(a):
  if a<0:
    return "Negative"
  elif a==0:
    return "Neutral"
  else:
    return "Positive"
data_score= data_polarity.apply(getTextAnalysis)

# to create the data frame
frame = { 'Tweets': data['tweets'], 'Polarity': data_polarity, 'Score' : data_score, 'Subjectivity' : data_subjectivity }
#Creating DataFrame by passing Dictionary
df = pd.DataFrame(frame)
#print(df.head)

#to get the percentage of positive tweets
positive = df[df['Score']=="Positive"]
print(str(positive.shape[0]/(df.shape[0])*100)+"% of positive tweets")
pos =positive.shape[0]/df.shape[0]*100

#to get the percentage of negative tweets
negative = df[df['Score']=="Negative"]
print(str(negative.shape[0]/(df.shape[0])*100)+"% of negative tweets")
pos = negative.shape[0]/df.shape[0]*100

#to get the percentage of neutral tweets
neutral = df[df['Score']=="Neutral"]
print(str(neutral.shape[0]/(df.shape[0])*100)+"% of neutral tweets")
pos = neutral.shape[0]/df.shape[0]*100

# plot bar graph 
labels=df.groupby('Score').count().index.values
values = df.groupby('Score').size().values
#plt.bar(labels,values)


In [ ]:
#bag of words
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(data['clean_tweets'])
vectorizer.get_feature_names_out()
vectorizer.vocabulary_

In [ ]:
#N-grams
import nltk
nltk.download('punkt')
TextBlob(data['clean_tweets'][0]).ngrams(2)

In [ ]:
#parts of speech tagging
import spacy
from spacy import displacy
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag, word_tokenize, RegexpParser
nlp = spacy.load("en_core_web_sm")
for i in df.index:
  doc = nlp(df.at[i,'Tweets'])
  count = doc.count_by(spacy.attrs.POS)
  for k,j in count.items():
    print(doc.vocab[k].text, "|",j)

  tagged = pos_tag(word_tokenize(df.at[0,'Tweets']))

  #Extract all parts of speech from any text
  chunker = RegexpParser("""
                    NP: {<DT>?<JJ>*<NN>} #To extract Noun Phrases
                    P: {<IN>}            #To extract Prepositions
                    V: {<V.*>}           #To extract Verbs
                    PP: {<p> <NP>}       #To extract Prepositional Phrases
                    VP: {<V> <NP|PP>*}   #To extract Verb Phrases
                    """)

  # Print all parts of speech in above sentence
  output = chunker.parse(tagged)
  print(output)
  displacy.render(doc, style='dep', jupyter=True, options={'distance': 100})

In [ ]:
#words embedding
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors # load the Stanford GloVe model
glove_input_file = '/content/gdrive/My Drive/glove.6B.100d.txt'
word2vec_output_file = '/content/gdrive/My Drive/glove.6B.100d.txt.word2vec'
glove_model = glove2word2vec(glove_input_file, word2vec_output_file)


filename = '/content/gdrive/My Drive/glove.6B.100d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)
model['go']
model['away']
(model['go'] + model['away'])/2

Question 3 (10 points): Use any of the feature selection methods mentioned in this paper "Deng, X., Li, Y., Weng, J., & Zhang, J. (2019). Feature selection for text classification: A review. Multimedia Tools & Applications, 78(3)." Select the most important features you extracted above, rank the features based on their importance in the descending order. 

In [ ]:
'''
The most important feature extracted in above data is sentiment lexicon as it gives the true emotion of tweets(positive, negative,netural)
ranking the features in descending order: 
1.sentiment Lexicon
2.words embedding
3.parts of speech tagging
4.N-grams
5.bag of words
# Since I didnot get access to the article mentioned above I made above statements by considering the materials on websites.
'''

Question 4 (10 points): Write python code to rank the text based on text similarity. Based on the text data you used for question 2, design a query to match the most relevant docments. Please use the BERT model to represent both your query and the text data, then calculate the cosine similarity between the query and each text in your data. Rank the similary with descending order. 

In [ ]:
#Finding the similarity between tweets
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer
sentences = df['Tweets']

model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(
    [sentence_embeddings[0]],
    sentence_embeddings[1:]
)

In [ ]:
from transformers import AutoTokenizer, AutoModel # to initialise model and tokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
sentences = df['Tweets']

tokens = {'input_ids': [], 'attention_mask': []}
for sentence in sentences:
    # encode each sentence and append to dictionary
    new_tokens = tokenizer.encode_plus(sentence, max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

# reformat list of tensors into single tensor
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
outputs = model(**tokens)
outputs.keys()

embeddings = outputs.last_hidden_state
embeddings
embeddings.shape

In [ ]:
attention_mask = tokens['attention_mask']
attention_mask.shape
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask.shape
mask


In [ ]:
masked_embeddings = embeddings * mask
masked_embeddings.shape
masked_embeddings

In [ ]:
summed = torch.sum(masked_embeddings, 1)
summed.shape
summed_mask = torch.clamp(mask.sum(1), min=1e-9)
summed_mask.shape
summed_mask
mean_pooled = summed / summed_mask
mean_pooled

In [ ]:
#as we get dense vector, cosine simialrity between tweets 
from sklearn.metrics.pairwise import cosine_similarity
mean_pooled = mean_pooled.detach().numpy()

# calculate
cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)